https://nces.ed.gov/ccd/elsi/tableGenerator.aspx

https://www.usinflationcalculator.com/inflation/current-inflation-rates/
https://data.bls.gov/pdq/SurveyOutputServlet

In [1]:
from collections import namedtuple

import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.regression

In [2]:
df = pl.read_csv(
    "./data/nces/geo/ELSI/state/ELSI_csv_export_6385899184786235507585.csv",
    skip_rows=6,
    n_rows=51
).drop("State Name [State] 2018-19").with_columns(
    pl.col("State Name").str.to_titlecase()
)
df = df.unpivot(
    on=df.columns[1:],
    index="State Name",
    variable_name="VARIABLE",
    value_name="VALUE"
).with_columns(
    SCHOOL_YEAR=pl.col("VARIABLE").str.extract(r"(\d{4}\-\d{2})$"),
    VARIABLE=pl.col("VARIABLE").str.replace(r" \[(State( Finance)?)\] (\d{4}\-\d{2})$", ""),
    VALUE=pl.col("VALUE").replace(chr(8211), None).str.to_integer(),
).pivot(
    on="VARIABLE",
    index=["State Name", "SCHOOL_YEAR"],
    values="VALUE",
    aggregate_function="first"
).with_columns(
    pl.sum_horizontal("Kindergarten Students", "Prekindergarten Students", "Grades 1-8 Students", "Grades 9-12 Students").alias("Total Students"),
    pl.sum_horizontal("Grades 1-8 Students", "Grades 9-12 Students").alias("Grades 1-12 Students")
).sort("State Name", "SCHOOL_YEAR").with_columns(
    pl.col("SCHOOL_YEAR").str.extract(r"^(\d{4})").str.to_integer().alias("SCHOOL_YEAR_START"),
    pl.col("SCHOOL_YEAR").str.extract(r"\-(\d{2})$").str.to_integer().add(2000).alias("SCHOOL_YEAR_END")
)
df

State Name,SCHOOL_YEAR,Grades 1-8 Students,Grade 8 Students,Grade 7 Students,Grade 6 Students,Grade 5 Students,Grade 4 Students,Grade 3 Students,Grade 2 Students,Grade 1 Students,Total Expenditures (TE11+E4D+E7A1),Total Revenues (TR) per Pupil (MEMBR),Local Revenues (STR1+R2) per Pupil (MEMBR),State Revenues (R3) per Pupil (MEMBR),Federal Revenues (STR4) per Pupil (MEMBR),Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR),Grades 9-12 Students,Prekindergarten Students,Kindergarten Students,Total Students,Grades 1-12 Students,SCHOOL_YEAR_START,SCHOOL_YEAR_END
str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Alabama""","""2011-12""",461122,57919,59050,59082,58415,56491,55991,56315,57859,7229298957,9534,3125,5284,1126,9709,217615,8282,57602,744621,678737,2011,2012
"""Alabama""","""2012-13""",458974,58151,59685,58240,56523,55859,55868,56361,58287,7438871954,9653,3225,5286,1142,9990,217203,9032,59428,744637,676177,2012,2013
"""Alabama""","""2013-14""",458327,58799,58455,56428,55699,55719,56003,56909,60315,7591336635,9913,3341,5448,1124,10173,218705,9415,59757,746204,677032,2013,2014
"""Alabama""","""2014-15""",454081,57471,56251,54859,55265,55479,56007,58370,60379,7616860366,9992,3321,5549,1122,10235,221068,11076,57939,744164,675149,2014,2015
"""Alabama""","""2015-16""",452764,55792,55172,54900,55340,55808,57963,58766,59023,7856051026,10200,3485,5578,1138,10562,222182,13230,55613,743789,674946,2015,2016
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Wyoming""","""2014-15""",58774,6927,6903,7181,7076,7438,7588,7835,7826,1929789830,20855,7697,11874,1284,20515,26732,564,7997,94067,85506,2014,2015
"""Wyoming""","""2015-16""",59453,6902,7191,7155,7451,7551,7839,7704,7660,2020050580,21569,7851,12414,1305,21327,26914,509,7841,94717,86367,2015,2016
"""Wyoming""","""2016-17""",58998,7135,7071,7352,7452,7642,7616,7422,7308,1913258414,20508,7128,12122,1258,20317,26924,664,7584,94170,85922,2016,2017


In [3]:
inflation = pl.read_excel(
    "./data/bls/inflation/SeriesReport-20240811173715_e1eda9.xlsx",
    read_options=dict(header_row=11),
)
inflation = inflation.with_columns(
    pl.col("Annual").shift(-1).alias("Annual (Next Year)")
).filter(pl.col("Year") < 2020).sort("Year", descending=True).with_columns(
    (1 + pl.col("Annual (Next Year)") / 100).cum_prod().alias("Current to 2020 Dollar Conversion Rate")
).sort("Year")
inflation

Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual,HALF1,HALF2,Annual (Next Year),Current to 2020 Dollar Conversion Rate
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2010,1.6,1.3,1.1,0.9,0.9,0.9,0.9,0.9,0.8,0.6,0.8,0.8,1.0,1.1,0.8,1.7,1.208257
2011,1.0,1.1,1.2,1.3,1.5,1.6,1.8,2.0,2.0,2.1,2.2,2.2,1.7,1.3,2.0,2.1,1.18806
2012,2.3,2.2,2.3,2.3,2.3,2.2,2.1,1.9,2.0,2.0,1.9,1.9,2.1,2.2,2.0,1.8,1.163624
2013,1.9,2.0,1.9,1.7,1.7,1.6,1.7,1.8,1.7,1.7,1.7,1.7,1.8,1.8,1.7,1.7,1.143049
2014,1.6,1.6,1.7,1.8,2.0,1.9,1.9,1.7,1.7,1.8,1.7,1.6,1.7,1.8,1.7,1.8,1.123942
2015,1.6,1.7,1.8,1.8,1.7,1.8,1.8,1.8,1.9,1.9,2.0,2.1,1.8,1.7,1.9,2.2,1.104069
2016,2.2,2.3,2.2,2.1,2.2,2.2,2.2,2.3,2.2,2.1,2.1,2.2,2.2,2.2,2.2,1.8,1.080302
2017,2.3,2.2,2.0,1.9,1.7,1.7,1.7,1.7,1.7,1.8,1.7,1.8,1.8,2.0,1.7,2.1,1.061201
2018,1.8,1.8,2.1,2.1,2.2,2.3,2.4,2.2,2.2,2.1,2.2,2.2,2.1,2.1,2.2,2.2,1.039374


In [4]:
student_investment = df.join(
    inflation.select("Year", "Current to 2020 Dollar Conversion Rate"),
    left_on="SCHOOL_YEAR_END",
    right_on="Year",
).with_columns(
    (
        pl.col("Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR)")
        * pl.col("Current to 2020 Dollar Conversion Rate")
    ).alias("Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR) (2020 Dollars)")
).with_columns(
    pl.col("Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR) (2020 Dollars)")
    .rolling_sum(4)
    .over("State Name")
    .alias("Total Expenditures (TE11+E4D+E7A1) per Graduating Pupil (MEMBR) (4-Year Rolling Sum) (2020 Dollars)"),
).select(
    [
        "State Name",
        "SCHOOL_YEAR",
        "Total Expenditures (TE11+E4D+E7A1) per Graduating Pupil (MEMBR) (4-Year Rolling Sum) (2020 Dollars)"
    ]
).filter(
    pl.col("SCHOOL_YEAR").is_in(["2014-15", "2018-19"])
).pivot(
    on="SCHOOL_YEAR",
    index="State Name",
    values="Total Expenditures (TE11+E4D+E7A1) per Graduating Pupil (MEMBR) (4-Year Rolling Sum) (2020 Dollars)",
).with_columns(
    (pl.col("2018-19") - pl.col("2014-15")).alias("Investment Change")
).sort("Investment Change")
student_investment

State Name,2014-15,2018-19,Investment Change
str,f64,f64,f64
"""Alaska""",94046.988007,81784.974805,-12262.013202
"""Wyoming""",86562.276805,82196.045171,-4366.231634
"""Louisiana""",54834.153752,52740.849105,-2093.304647
"""West Virginia""",55539.244061,53498.039359,-2041.204702
"""Arkansas""",50161.688322,48975.172334,-1186.515988
…,…,…,…
"""Hawaii""",60234.237008,68145.601147,7911.364139
"""New York""",99396.422089,107649.308984,8252.886895
"""Oregon""",50202.867182,59267.572773,9064.705592


In [5]:
math_performance = pl.concat(
    [
        pl.read_csv(
            "data/nces/geo/nations-report-card/2019/state_grade-8_math_SPCsv202408081556.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2015).alias("Graduating Year")
        ),
        pl.read_csv(
            "data/nces/geo/nations-report-card/2015/state_grade-8_math_SPCsv202408081407.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2019).alias("Graduating Year"),
            pl.col("AP").replace("#", None).str.to_integer()
        ),
    ]
).rename(
    {
        "Jurisdiction": "State Name",
        "MN": "Mean Grade 8 Math Score",
        "AB": "At or Above Basic (%)",
        "AP": "At or Above Proficient (%)",
    }
).pivot(
    on="Graduating Year",
    index="State Name",
    values=["Mean Grade 8 Math Score", "At or Above Basic (%)", "At or Above Proficient (%)"],
).with_columns(
    (pl.col("Mean Grade 8 Math Score_2019") - pl.col("Mean Grade 8 Math Score_2015")).alias("Mean Grade 8 Math Score Change"),
    (pl.col("At or Above Basic (%)_2019") - pl.col("At or Above Basic (%)_2015")).alias("At or Above Basic (%) Change"),
    (pl.col("At or Above Proficient (%)_2019") - pl.col("At or Above Proficient (%)_2015")).alias("At or Above Proficient (%) Change"),
).select(
    "State Name",
    "Mean Grade 8 Math Score Change",
    "At or Above Basic (%) Change",
    "At or Above Proficient (%) Change",
).sort("Mean Grade 8 Math Score Change")

math_performance

State Name,Mean Grade 8 Math Score Change,At or Above Basic (%) Change,At or Above Proficient (%) Change
str,i64,i64,i64
"""District of Columbia""",-6,-4,-4
"""Florida""",-4,-2,-5
"""Louisiana""",-4,-4,-5
"""North Carolina""",-3,-2,-4
"""Mississippi""",-3,-2,-2
…,…,…,…
"""Texas""",4,7,2
"""Hawaii""",4,5,2
"""Rhode Island""",5,8,3


In [6]:
Result = namedtuple("CorrelationResult", ["result", "r_squared", "p_value", "coef"])

def correlate(df: pl.DataFrame, x: str, y: str, show_summary: bool = True) -> Result:
    """Return the OLS Adjusted R-squared and p-value of the model."""
    result = statsmodels.regression.linear_model.OLS(
        endog=df[y].to_list(),
        exog=df[x].to_list(),
    ).fit()
    if show_summary:
        display(result.summary())
    return Result(
        result=result,
        r_squared=float(result.rsquared_adj),
        p_value=float(result.pvalues[0]),
        coef=float(result.params[0]),
    )

In [7]:

def generate_state_investment_and_math_performance_bar_chart(
    column: str, legend_title: str, description: str, change_measure: str
) -> go.Figure:
    df = student_investment.join(
        math_performance,
        on="State Name"
    ).sort("Investment Change")
    result = correlate(df, "Investment Change", column)
    return px.bar(
        df,
        x="State Name",
        y="Investment Change",
        color=column,
        title=f"US Primary Education Return on Investment<br><sup><i>{description}</i></sup>",
        template="plotly_dark",
        color_continuous_scale=px.colors.sequential.RdBu,
        labels={
            "State Name": "State",
            "Investment Change": "4-Year Expenditure Change (2020 Dollars)",
            column: f"Change in NAEP {legend_title}",
        },
        width=1000,
        height=600,
    ).update_layout(
        xaxis=dict(showticklabels=False, title=None),
        yaxis_title="Per Pupil, 4-Year Expenditure Change Between 2015 and 2019<br>(2020 Dollars)",
        yaxis_title_font_size=10,
        coloraxis_colorbar_title=f"Change in <i>NAEP {legend_title}</i>",
        coloraxis_colorbar_orientation="h",
        plot_bgcolor="#171717",
        paper_bgcolor="#171717",
    ).update_coloraxes(
        cmid=0,
    ).add_annotation(
        x=1.07,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Chart by Dominic Tarro | 𝕏 @dominictarro",
                "Sources: NAEP Mathematics Assessments (2015, 2019)",
                "CCD National Public Education Financial Survey (2010-2019)"
            )
        ),
        align="right",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    ).add_annotation(
        x=-0.08,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Note: The NAEP Mathematics scale ranges from 0 to 500.",
                "<i>Per Pupil, 4-Year Expenditure</i> is an aggregate from 5th-8th grade. Dollar adjustment was made using <br>each school year's ending year.",
                f"OLS Adj. R-squared: {result.r_squared:.03f}; P|t|>{result.p_value:.03f}; {change_measure.title()} change per $1000 change in per student spending={result.coef * 1000:.2f}"
            )
        ),
        align="left",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    )

_fig1 = generate_state_investment_and_math_performance_bar_chart(
    "Mean Grade 8 Math Score Change",
    "Average Math Score",
    "States spending more per student from 5th through 8th grade had a negligible relationship with average 8th grade math performance.",
    "score",
)
_fig1.write_image("charts/spending-school-roi-math-mean.png")
_fig1.show()
_fig2 = generate_state_investment_and_math_performance_bar_chart(
    "At or Above Basic (%) Change",
    "At or Above Basic</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a small relationship with basic 8th grade math performance.",
    "score",
)
_fig2.write_image("charts/spending-school-roi-math-basic.png")
_fig2.show()
_fig3 = generate_state_investment_and_math_performance_bar_chart(
    "At or Above Proficient (%) Change",
    "At or Above Proficient</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a negligible relationship with 8th grade math profieciency.",
    "score",
)
_fig3.write_image("charts/spending-school-roi-math-proficient.png")
_fig3.show()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.024
Model:                            OLS   Adj. R-squared (uncentered):              0.004
Method:                 Least Squares   F-statistic:                              1.220
Date:                Sun, 25 Aug 2024   Prob (F-statistic):                       0.275
Time:                        23:38:17   Log-Likelihood:                         -118.33
No. Observations:                  50   AIC:                                      238.7
Df Residuals:                      49   BIC:                                      240.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          9.337e-05   8.45e-05      1.105      0.275   -7.65e-05       0.000
==============================================================================
Omnibus:                        1.122   Durbin-Watson:                   2.075
Prob(Omnibus):                  0.571   Jarque-Bera (JB):                0.986
Skew:                           0.333   Prob(JB):                        0.611
Kurtosis:                       2.827   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.152
Model:                            OLS   Adj. R-squared (uncentered):              0.135
Method:                 Least Squares   F-statistic:                              8.805
Date:                Sun, 25 Aug 2024   Prob (F-statistic):                     0.00464
Time:                        23:38:18   Log-Likelihood:                         -133.08
No. Observations:                  50   AIC:                                      268.2
Df Residuals:                      49   BIC:                                      270.1
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0003      0.000      2.967      0.005       0.000       0.001
==============================================================================
Omnibus:                        9.470   Durbin-Watson:                   1.120
Prob(Omnibus):                  0.009   Jarque-Bera (JB):                9.526
Skew:                           0.783   Prob(JB):                      0.00854
Kurtosis:                       4.457   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):             -0.019
Method:                 Least Squares   F-statistic:                            0.04891
Date:                Sun, 25 Aug 2024   Prob (F-statistic):                       0.826
Time:                        23:38:18   Log-Likelihood:                         -124.54
No. Observations:                  50   AIC:                                      251.1
Df Residuals:                      49   BIC:                                      253.0
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          2.117e-05   9.57e-05      0.221      0.826      -0.000       0.000
==============================================================================
Omnibus:                        1.633   Durbin-Watson:                   2.349
Prob(Omnibus):                  0.442   Jarque-Bera (JB):                1.609
Skew:                           0.378   Prob(JB):                        0.447
Kurtosis:                       2.553   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
reading_performance = pl.concat(
    [
        pl.read_csv(
            "data/nces/geo/nations-report-card/2019/state_grade-8_reading_SPCsv202408081536.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2015).alias("Graduating Year")
        ),
        pl.read_csv(
            "data/nces/geo/nations-report-card/2015/state_grade-8_reading_SPCsv202408081449.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2019).alias("Graduating Year"),
        ),
    ]
).with_columns(
            pl.col("MN").replace(chr(8212), None).str.to_integer(),
            pl.col("AB").replace(chr(8212), None).str.to_integer(),
            pl.col("AP").replace(chr(8212), None).str.to_integer(),

).rename(
    {
        "Jurisdiction": "State Name",
        "MN": "Mean Grade 8 Reading Score",
        "AB": "At or Above Basic (%)",
        "AP": "At or Above Proficient (%)",
    }
).pivot(
    on="Graduating Year",
    index="State Name",
    values=["Mean Grade 8 Reading Score", "At or Above Basic (%)", "At or Above Proficient (%)"],
).with_columns(
    (pl.col("Mean Grade 8 Reading Score_2019") - pl.col("Mean Grade 8 Reading Score_2015")).alias("Mean Grade 8 Reading Score Change"),
    (pl.col("At or Above Basic (%)_2019") - pl.col("At or Above Basic (%)_2015")).alias("At or Above Basic (%) Change"),
    (pl.col("At or Above Proficient (%)_2019") - pl.col("At or Above Proficient (%)_2015")).alias("At or Above Proficient (%) Change"),
).select(
    "State Name",
    "Mean Grade 8 Reading Score Change",
    "At or Above Basic (%) Change",
    "At or Above Proficient (%) Change",
).sort("Mean Grade 8 Reading Score Change")

reading_performance

State Name,Mean Grade 8 Reading Score Change,At or Above Basic (%) Change,At or Above Proficient (%) Change
str,i64,i64,i64
"""Puerto Rico""",null,null,null
"""Mississippi""",-4,-4,-5
"""DoDEA""",-3,-1,-5
"""North Carolina""",-2,0,-3
"""Louisiana""",-2,-2,-4
…,…,…,…
"""Minnesota""",6,7,6
"""Iowa""",6,8,3
"""Alabama""",6,7,2


In [9]:
def generate_state_investment_and_reading_performance_bar_chart(
    column: str, legend_title: str, description: str, change_measure: str
) -> go.Figure:
    df = student_investment.join(
        reading_performance,
        on="State Name"
    ).sort("Investment Change")
    result = correlate(df, "Investment Change", column)
    return px.bar(
        df,
        x="State Name",
        y="Investment Change",
        color=column,
        title=f"US Primary Education Return on Investment<br><sup><i>{description}</i></sup>",
        template="plotly_dark",
        color_continuous_scale=px.colors.sequential.RdBu,
        labels={
            "State Name": "State",
            "Investment Change": "4-Year Expenditure Change (2020 Dollars)",
            column: f"Change in NAEP {legend_title}",
        },
        width=1000,
        height=600,
    ).update_layout(
        xaxis=dict(showticklabels=False, title=None),
        yaxis_title="Per Pupil, 4-Year Expenditure Change Between 2015 and 2019<br>(2020 Dollars)",
        yaxis_title_font_size=10,
        coloraxis_colorbar_title=f"Change in <i>NAEP {legend_title}</i>",
        coloraxis_colorbar_orientation="h",
        plot_bgcolor="#171717",
        paper_bgcolor="#171717",
    ).update_coloraxes(
        cmid=0,
    ).add_annotation(
        x=1.07,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Chart by Dominic Tarro | 𝕏 @dominictarro",
                "Sources: NAEP Reading Assessments (2015, 2019)",
                "CCD National Public Education Financial Survey (2010-2019)"
            )
        ),
        align="right",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    ).add_annotation(
        x=-0.08,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "The NAEP Reading scale ranges from 0 to 500.",
                "<i>Per Pupil, 4-Year Expenditure</i> is an aggregate from 5th-8th grade. Dollar adjustment was made using <br>each school year's ending year.",
                f"OLS Adj. R-squared: {result.r_squared:.03f}; P|t|>{result.p_value:.03f}; {change_measure.title()} change per $1000 in per student spending={result.coef * 1000:.2f}"
            )
        ),
        align="left",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    )

_fig4 = generate_state_investment_and_reading_performance_bar_chart(
    "Mean Grade 8 Reading Score Change",
    "Average Reading Score",
    "States spending more per student from 5th through 8th grade had a negligible relationship with average 8th grade reading performance.",
    "% pts",
)
_fig4.write_image("./charts/spending-school-roi-reading-mean.png")
_fig4.show()
_fig5 = generate_state_investment_and_reading_performance_bar_chart(
    "At or Above Basic (%) Change",
    "At or Above Basic</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a small relationship with basic 8th grade reading performance.",
    "score",
)
_fig5.write_image("./charts/spending-school-roi-reading-basic.png")
_fig5.show()
_fig6 = generate_state_investment_and_reading_performance_bar_chart(
    "At or Above Proficient (%) Change",
    "At or Above Proficient</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a negligible relationship with 8th grade reading proficiency.",
    "% pts",
)
_fig6.write_image("./charts/spending-school-roi-reading-proficient.png")
_fig6.show()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.063
Model:                            OLS   Adj. R-squared (uncentered):              0.044
Method:                 Least Squares   F-statistic:                              3.303
Date:                Sun, 25 Aug 2024   Prob (F-statistic):                      0.0753
Time:                        23:38:18   Log-Likelihood:                         -134.79
No. Observations:                  50   AIC:                                      271.6
Df Residuals:                      49   BIC:                                      273.5
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0002      0.000      1.817      0.075   -2.26e-05       0.000
==============================================================================
Omnibus:                        0.759   Durbin-Watson:                   0.950
Prob(Omnibus):                  0.684   Jarque-Bera (JB):                0.205
Skew:                           0.071   Prob(JB):                        0.903
Kurtosis:                       3.279   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.123
Model:                            OLS   Adj. R-squared (uncentered):              0.105
Method:                 Least Squares   F-statistic:                              6.847
Date:                Sun, 25 Aug 2024   Prob (F-statistic):                      0.0118
Time:                        23:38:18   Log-Likelihood:                         -143.31
No. Observations:                  50   AIC:                                      288.6
Df Residuals:                      49   BIC:                                      290.5
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0004      0.000      2.617      0.012    8.46e-05       0.001
==============================================================================
Omnibus:                        2.941   Durbin-Watson:                   0.632
Prob(Omnibus):                  0.230   Jarque-Bera (JB):                2.294
Skew:                           0.131   Prob(JB):                        0.318
Kurtosis:                       4.016   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):             -0.020
Method:                 Least Squares   F-statistic:                            0.02618
Date:                Sun, 25 Aug 2024   Prob (F-statistic):                       0.872
Time:                        23:38:18   Log-Likelihood:                         -124.49
No. Observations:                  50   AIC:                                      251.0
Df Residuals:                      49   BIC:                                      252.9
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.547e-05   9.56e-05      0.162      0.872      -0.000       0.000
==============================================================================
Omnibus:                        0.343   Durbin-Watson:                   1.731
Prob(Omnibus):                  0.843   Jarque-Bera (JB):                0.239
Skew:                           0.163   Prob(JB):                        0.887
Kurtosis:                       2.908   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
"${:,.2f}".format(
    student_investment.join(
        df.filter(pl.col("SCHOOL_YEAR") == "2018-19").select("State Name", "Grade 8 Students"),
        on="State Name"
    ).with_columns(
        (pl.col("Investment Change") * pl.col("Grade 8 Students")).alias("Total Investment Change"),
    )["Total Investment Change"].sum()
)

'$14,141,710,573.34'

In [11]:
math_performance["Mean Grade 8 Math Score Change"].mean(), reading_performance["Mean Grade 8 Reading Score Change"].mean()

(0.6296296296296297, 2.509433962264151)

In [12]:
math_performance["At or Above Basic (%) Change"].mean(), reading_performance["At or Above Basic (%) Change"].mean()

(2.3518518518518516, 3.6226415094339623)

In [13]:
math_performance["At or Above Proficient (%) Change"].mean(), reading_performance["At or Above Proficient (%) Change"].mean()

(-0.32075471698113206, 0.6415094339622641)